# air_passengers ramp kit - feature enrichment and extractor construction
<i>Sylvain Tostain, 2017</i>

# Introduction
The aim of this notebook is to provide and document a relevant feature extractor for the air_passenger RAMP kit.

As introduced in the starting kit notebook, a good feature extractor is of particular relevance in this ramp kit due to the fact that the data provided is rather thin, and that we noticed in exploratory visualisations that the data seems to expose seasonality and possible special causes that ought to be understood and captured before training a model.

At first, let's improt and have a look at the dataset provided.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display

## Fetching RAMP dataset to load it in a dataframe with pandas

In [2]:
data = pd.read_csv("../data/train.csv.bz2")

The data made available are as follows, `log_PAX` being our labels.

In [3]:
data.dtypes

DateOfDeparture      object
Departure            object
Arrival              object
WeeksToDeparture    float64
log_PAX             float64
std_wtd             float64
dtype: object

And an overview of the contents.

In [4]:
data.head(5)

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159


Let's have a look at the timeframe we are adressing. This is especially usefull for the sake of data enrichment from other sources.

In [5]:
print(min(data['DateOfDeparture']))
print(max(data['DateOfDeparture']))

2011-09-01
2013-03-05


## Fetching the additional external dataset provided with the RAMP kit
The dataset comes in this RAMP kit with a sample additional dataset `external_data.csv` pertaining weather data and used as sample in the kit demonstration. This data might come in handy as well, we'll take it on board too as a seed for our own `external_data.csv`.

In [6]:
ext_data = pd.read_csv("../submissions/starting_kit/external_data.csv")
ext_data.dtypes

Date                           object
AirPort                        object
Max TemperatureC                int64
Mean TemperatureC               int64
Min TemperatureC                int64
Dew PointC                      int64
MeanDew PointC                  int64
Min DewpointC                   int64
Max Humidity                    int64
Mean Humidity                   int64
Min Humidity                    int64
Max Sea Level PressurehPa       int64
Mean Sea Level PressurehPa      int64
Min Sea Level PressurehPa       int64
Max VisibilityKm                int64
Mean VisibilityKm               int64
Min VisibilitykM                int64
Max Wind SpeedKm/h              int64
Mean Wind SpeedKm/h             int64
Max Gust SpeedKm/h            float64
Precipitationmm                object
CloudCover                      int64
Events                         object
WindDirDegrees                  int64
dtype: object

These are meteo descriptors.

We'll keep them but this needs extra attention on complimentary data to enrich our dataset, as it is believed to be a key success factor...

In [7]:
ext_data.head(5)

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149


We notice that Events may require one hot encoding, but this is not the place for further data exploration. Please see the dedicated notebook if necessary.

In [8]:
ext_data['Events'] = ext_data['Events'].fillna('None')
print("Factors in Events:")
print(ext_data['Events'].unique())
print("Total number of factors in Events: {}".format(len(ext_data['Events'].unique())))

Factors in Events:
['None' 'Rain' 'Rain-Thunderstorm' 'Fog' 'Fog-Rain-Thunderstorm'
 'Rain-Snow' 'Snow' 'Fog-Rain' 'Thunderstorm' 'Fog-Snow' 'Fog-Rain-Snow'
 'Fog-Rain-Snow-Thunderstorm' 'Rain-Snow-Thunderstorm'
 'Rain-Hail-Thunderstorm' 'Fog-Rain-Hail-Thunderstorm'
 'Rain-Thunderstorm-Tornado']
Total number of factors in Events: 16


This kind of data might happen to be significant, unfortunately it is very inconveniently structured.

More sophisticated encoding could be considered at a later stage, but we'll start with a simple one hot encoding.

In [9]:
ext_data_enc = ext_data

ext_data_enc = ext_data_enc.join(pd.get_dummies(ext_data_enc['Events']))
ext_data_enc = ext_data_enc.drop('Events', axis=1)

# Also needed later for a graceful join with other data...
ext_data_enc['Date'] = pd.to_datetime(ext_data_enc['Date'])

ext_data_enc.head(5)

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,Fog,Fog-Rain,Fog-Rain-Hail-Thunderstorm,Fog-Rain-Snow,Fog-Rain-Snow-Thunderstorm,Fog-Rain-Thunderstorm,Fog-Snow,None,Rain,Rain-Hail-Thunderstorm,Rain-Snow,Rain-Snow-Thunderstorm,Rain-Thunderstorm,Rain-Thunderstorm-Tornado,Snow,Thunderstorm
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,129,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,185,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,147,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,139,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,149,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## Structuring the base dataset
As seen in the visualisations and in the starting kit, it is relevant to provide some structure to the base dataset, more specifically:
* Regarding dates, in order to allow our models to capture seasonality. Months, weeks and weekdays are probably the most relevant. We can forget about days of the month that are probably less relevant.
* Regarding factors, we'll apply one hot encoding on departure and arrival airports.

Nevertheless, we have to keep in mind that there will be a huge cost in termes of dimensions...

In [10]:
print("Factors in Departure:")
print(data['Departure'].unique())
print("Total number of factors in Departure: {}".format(len(data['Departure'].unique())))

print("\nFactors in Arrival:")
print(data['Arrival'].unique())
print("Total number of factors in Arrival: {}".format(len(data['Arrival'].unique())))

Factors in Departure:
['ORD' 'LAS' 'DEN' 'ATL' 'SFO' 'EWR' 'IAH' 'LAX' 'DFW' 'SEA' 'JFK' 'PHL'
 'MIA' 'DTW' 'BOS' 'MSP' 'CLT' 'MCO' 'PHX' 'LGA']
Total number of factors in Departure: 20

Factors in Arrival:
['DFW' 'DEN' 'LAX' 'ORD' 'SFO' 'MCO' 'LAS' 'CLT' 'MSP' 'EWR' 'PHX' 'DTW'
 'MIA' 'BOS' 'PHL' 'JFK' 'ATL' 'LGA' 'SEA' 'IAH']
Total number of factors in Arrival: 20


Let's apply these transformations.

We'll keep weekdays and weeks, and forget about the other time series factors so far, as we see lesser interest from our visualisations.

In [11]:
# Directly inspired from the starting kit notebook.
data_enc = data

# One-hot encoding of departure points, then drop of the initial feature
data_enc = data_enc.join(pd.get_dummies(data_enc['Departure'], prefix='d'))
data_enc = data_enc.drop('Departure', axis=1)

# One-hot encoding of arrival points, then drop of the initial feature
data_enc = data_enc.join(pd.get_dummies(data_enc['Arrival'], prefix='a'))
data_enc = data_enc.drop('Arrival', axis=1)

# One-hot encoding of temporal variables that might catch seasonalities and/or special causes
# following http://stackoverflow.com/questions/16453644/regression-with-date-variable-using-scikit-learn
data_enc['DateOfDeparture'] = pd.to_datetime(data_enc['DateOfDeparture'])

data_enc['weekday'] = data_enc['DateOfDeparture'].dt.weekday
data_enc = data_enc.join(pd.get_dummies(data_enc['weekday'], prefix='wd'))
data_enc = data_enc.drop('weekday', axis=1)

data_enc['week'] = data_enc['DateOfDeparture'].dt.week
data_enc = data_enc.join(pd.get_dummies(data_enc['week'], prefix='w'))
data_enc = data_enc.drop('week', axis=1)

# Commented out : probably useless and most certainly costly in terms of dimensions...
#
# data_enc['year'] = data_enc['DateOfDeparture'].dt.year
# data_enc = data_enc.join(pd.get_dummies(data_enc['year'], prefix='y'))
#
# data_enc['month'] = data_enc['DateOfDeparture'].dt.month
# data_enc = data_enc.join(pd.get_dummies(data_enc['month'], prefix='m'))
#
# data_enc['day'] = data_enc['DateOfDeparture'].dt.day
# data_enc = data_enc.join(pd.get_dummies(data_enc['day'], prefix='d'))
#
# data_enc['n_days'] = data_enc['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

As a result...

In [12]:
print(list(data_enc.columns))
print("Total number of columns: {}".format(len(list(data_enc.columns))))

['DateOfDeparture', 'WeeksToDeparture', 'log_PAX', 'std_wtd', 'd_ATL', 'd_BOS', 'd_CLT', 'd_DEN', 'd_DFW', 'd_DTW', 'd_EWR', 'd_IAH', 'd_JFK', 'd_LAS', 'd_LAX', 'd_LGA', 'd_MCO', 'd_MIA', 'd_MSP', 'd_ORD', 'd_PHL', 'd_PHX', 'd_SEA', 'd_SFO', 'a_ATL', 'a_BOS', 'a_CLT', 'a_DEN', 'a_DFW', 'a_DTW', 'a_EWR', 'a_IAH', 'a_JFK', 'a_LAS', 'a_LAX', 'a_LGA', 'a_MCO', 'a_MIA', 'a_MSP', 'a_ORD', 'a_PHL', 'a_PHX', 'a_SEA', 'a_SFO', 'wd_0', 'wd_1', 'wd_2', 'wd_3', 'wd_4', 'wd_5', 'wd_6', 'w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 'w_7', 'w_8', 'w_9', 'w_10', 'w_11', 'w_12', 'w_13', 'w_14', 'w_15', 'w_16', 'w_17', 'w_18', 'w_19', 'w_20', 'w_21', 'w_22', 'w_23', 'w_24', 'w_25', 'w_26', 'w_27', 'w_28', 'w_29', 'w_30', 'w_31', 'w_32', 'w_33', 'w_34', 'w_35', 'w_36', 'w_37', 'w_38', 'w_39', 'w_40', 'w_41', 'w_42', 'w_43', 'w_44', 'w_45', 'w_46', 'w_47', 'w_48', 'w_49', 'w_50', 'w_51', 'w_52']
Total number of columns: 103


At this stage, our base dataset is prepared and ready for enrichment...

## Creating a richer external dataset
### Introduction: data regarding public holidays
Given the observations made through visualisations, we have collected additional data.

Regarding holidays, the situation is rather complex given the fact that there is no regulated paid off days for every employer in the US. Even federal holidays are left to the discretion of the employers. We therefore adopted the following approach.

So far, the following data have been captured in an excel file:
* Statistics on paid off days, collected in 2011 by SHRM in <a href=https://www.shrm.org/hr-today/news/hr-news/Pages/paidholidaysin2011.aspx>this article</a>. Note: observances for which paid off days have been given by less than 10% of responding companies have not been retained as relevant.
* All federal holidays plus observances retained as significant for off days, as given on <a href=https://www.timeanddate.com/holidays/us/>timeanddate.com</a>

A holidays.csv file has been generated in which we aggregated, for the time period at hand:
* Date: the date;
* FederalHoliday: 1 if the day is a Federal Holiday, otherwise 0;
* PaidHoliday: a float between 0 and 1, capturing the proportion of respondents in SHRM article having given a paid off day (remember however that events with lesser than 10% paid off days have been ignored);
* Event: a text factor, describing the nature of the event taken into consideration.
Please note that Federal Holidays on sundays are usually observed on mondays, this has been taken into account.

Let's load the dataset.
### Loading holidays.csv

In [13]:
ext_holidays = pd.read_csv("../data_sources/holidays.csv")
ext_holidays.dtypes

Date               object
FederalHoliday      int64
PaidHoliday       float64
Event              object
dtype: object

In [14]:
print("Factors in Event:")
print(ext_holidays['Event'].unique())
print("Total number of factors in Event: {}".format(len(ext_holidays['Event'].unique())))
ext_holidays.head(5)

Factors in Event:
[nan 'labor' 'colombus' 'veterans' 'thanksgiving' 'thanksgiving_after'
 'christmas_eve_before' 'christmas_eve' 'christmas' 'new_year_eve_before'
 'new_year_eve' 'new_year' 'mlk_birthday' 'presidents' 'good_friday'
 'memorial' 'independence']
Total number of factors in Event: 17


,Date,FederalHoliday,PaidHoliday,Event
0,01/09/2011,0,0.00,NaN
1,02/09/2011,0,0.00,NaN
2,03/09/2011,0,0.00,NaN
3,04/09/2011,0,0.00,NaN
4,05/09/2011,1,0.95,labor


We need to turn the date into something useable for a join, and consider one-hot encoding for events (if we keep them).
### Encoding parameters from holidays properly

In [15]:
ext_holidays_enc = ext_holidays
ext_holidays_enc = ext_holidays_enc.join(pd.get_dummies(ext_holidays_enc['Event']))
ext_holidays_enc = ext_holidays_enc.drop('Event', axis=1)

ext_holidays_enc['Date'] = pd.to_datetime(ext_holidays_enc['Date'], dayfirst=True)

display(ext_holidays_enc.dtypes)
ext_holidays_enc.head(5)

Date                    datetime64[ns]
FederalHoliday                   int64
PaidHoliday                    float64
christmas                        uint8
christmas_eve                    uint8
christmas_eve_before             uint8
colombus                         uint8
good_friday                      uint8
independence                     uint8
labor                            uint8
memorial                         uint8
mlk_birthday                     uint8
new_year                         uint8
new_year_eve                     uint8
new_year_eve_before              uint8
presidents                       uint8
thanksgiving                     uint8
thanksgiving_after               uint8
veterans                         uint8
dtype: object

,Date,FederalHoliday,PaidHoliday,christmas,christmas_eve,christmas_eve_before,colombus,good_friday,independence,labor,memorial,mlk_birthday,new_year,new_year_eve,new_year_eve_before,presidents,thanksgiving,thanksgiving_after,veterans
0,2011-09-01,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2011-09-02,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2011-09-03,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2011-09-04,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2011-09-05,1,0.95,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


Our `ext_holidays_enc` dataframe is now ready for a join with the rest of `external_data_enc`.

### Joining weather data and public holiday data

In [16]:
#Date columns have the same name in both dataframe, the join is straigthforward
ext_full = pd.merge(ext_data_enc, ext_holidays_enc, how='left', left_on=['Date'], right_on=['Date'], sort=False)
ext_full.head(5)

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,Fog,Fog-Rain,Fog-Rain-Hail-Thunderstorm,Fog-Rain-Snow,Fog-Rain-Snow-Thunderstorm,Fog-Rain-Thunderstorm,Fog-Snow,None,Rain,Rain-Hail-Thunderstorm,Rain-Snow,Rain-Snow-Thunderstorm,Rain-Thunderstorm,Rain-Thunderstorm-Tornado,Snow,Thunderstorm,FederalHoliday,PaidHoliday,christmas,christmas_eve,christmas_eve_before,colombus,good_friday,independence,labor,memorial,mlk_birthday,new_year,new_year_eve,new_year_eve_before,presidents,thanksgiving,thanksgiving_after,veterans
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,129,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,185,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,147,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,139,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,149,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.95,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


We now have an enriched external dataset

## Wrap-up
We have build a first enriched dataset, that we'll test as a new submission : `iteration_0`

An empty directory has been created accordingly (if not, it needs to be on your local environment).

### 0. Scrapper
In addition to the two mandatory functions on RAMP, we captured steps necessary to produce the `external_data.csv` file in an additional `scrapper` function stored in the following cell.

In [17]:
import os

def scrapper(path):
    """This function combine data from the initial external_data.csv
    of the starting_kit together with additional data sources and
    serialise the dataframe in external.data.csv in the target directory.
    
    Argument:
    path : directory where to save external_data.csv"""

    # Import weather data from the original external_data.csv file
    weather = pd.read_csv('../submissions/starting_kit/external_data.csv')   
    # Import holiday data from the holiday.csv file 
    holidays = pd.read_csv('../data_sources/holidays.csv')
    
    # Prepare the key (Date columns) for the merge
    weather['Date'] = pd.to_datetime(weather['Date'])
    holidays['Date'] = pd.to_datetime(holidays['Date'], dayfirst=True)
    
    # Merge the dataframes
    ext_data = pd.merge(weather, holidays,
                        how='left',
                        left_on=['Date'],
                        right_on=['Date'],
                        sort=False)
    
    # Serialise ext_data to pack it into external_data.csv accepted by the RAMP
    ext_data.to_csv(os.path.join(path, 'external_data.csv'))

Running the following cell executes the scrapper and generates `external_data.csv` in the proper location.

In [18]:
scrapper('../submissions/iteration_0')

### 1. Feature extractor

We then propose the following feature extractor.

Nevertheless, for some unidentified reason, generating one hot encoding in meteo or holiday events causes the base regressor to throw an error, we deactivated this temporarily.

In [19]:
%%file ../submissions/iteration_0/feature_extractor.py
import pandas as pd
import os

# Inspired form the feature extractor that comes with starting_kit.
class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df

        
        # Fetches external data from external_data.csv
        path = os.path.dirname(__file__)
        ext_data = pd.read_csv(os.path.join(path, 'external_data.csv'))
        X_ext_data = ext_data[['Date', 'AirPort', 'Max TemperatureC',
                               'PaidHoliday', 'FederalHoliday']]
        
        # Merges (left join) fetched external data with base data
        X_ext_data = X_ext_data.rename(
            columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})
        X_encoded = pd.merge(
            X_encoded, X_ext_data,
            how='left',
            left_on=['DateOfDeparture', 'Arrival'],
            right_on=['DateOfDeparture', 'Arrival'],
            sort=False)
        
        # Creates one hot encoding for Departure, then drop the original feature
        X_encoded = X_encoded.join(pd.get_dummies(
            X_encoded['Departure'], prefix='d'))
        X_encoded = X_encoded.drop('Departure', axis=1)
        
        # Creates one hot encoding for Arrival, then drop the original feature
        X_encoded = X_encoded.join(pd.get_dummies(
            X_encoded['Arrival'], prefix='a'))
        X_encoded = X_encoded.drop('Arrival', axis=1)
        
        #FIXME : for some reason, these treatment raise an error in the regressor like this:
        #ValueError: Number of features of the model must match the input.
        #Model n_features is 137 and input n_features is 135
        #
        # Creates one hot encoding for meteo Events, then drop the original feature
        #X_encoded['Events'] = X_encoded['Events'].fillna('None')
        #X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Events'], prefix='mevent'))
        #X_encoded = X_encoded.drop('Events', axis=1)
        #
        # Creates one hot encoding for Holiday Events, then drop the original feature
        #X_encoded['Event'] = X_encoded['Event'].fillna('Ordinary')
        #X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Event'], prefix='h'))
        #X_encoded = X_encoded.drop('Event', axis=1)
        
        # Creates one hot encoding for time period likely to catch seasonality
        X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
        
        X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
        X_encoded = X_encoded.drop('weekday', axis=1)
        
        X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))
        X_encoded = X_encoded.drop('week', axis=1)
        
        # Drops DateOfDeparture
        X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
        
        # Return the values
        X_array = X_encoded.values
        return X_array

Overwriting ../submissions/iteration_0/feature_extractor.py


### 2. Regressor

In [20]:
%%file ../submissions/iteration_0/regressor.py
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator


class Regressor(BaseEstimator):
    def __init__(self):
        self.clf = RandomForestRegressor(
            n_estimators=10, max_depth=10, max_features=10)

    def fit(self, X, y):
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)

Overwriting ../submissions/iteration_0/regressor.py


### 3. Testing!

In [21]:
!cd .. & ramp_test_submission --submission=iteration_0

Testing Number of air passengers prediction
Reading train and test files from ./data ...
Reading cv ...
Training .\submissions\iteration_0 ...
CV fold 0
	train rmse = 0.683
	valid rmse = 0.747
	test rmse = 0.746
CV fold 1
	train rmse = 0.699
	valid rmse = 0.753
	test rmse = 0.762
CV fold 2
	train rmse = 0.678
	valid rmse = 0.73
	test rmse = 0.739
CV fold 3
	train rmse = 0.682
	valid rmse = 0.763
	test rmse = 0.76
CV fold 4
	train rmse = 0.69
	valid rmse = 0.762
	test rmse = 0.764
CV fold 5
	train rmse = 0.681
	valid rmse = 0.754
	test rmse = 0.751
CV fold 6
	train rmse = 0.731
	valid rmse = 0.794
	test rmse = 0.798
CV fold 7
	train rmse = 0.68
	valid rmse = 0.712
	test rmse = 0.726
----------------------------
Mean CV scores
----------------------------
train rmse = 0.691 ± 0.017
valid rmse = 0.752 ± 0.023
test rmse = 0.756 ± 0.02
----------------------------
Bagged scores
----------------------------
valid rmse = 0.737
test rmse = 0.741


At this stage, this provide only a very small improvement with regards to the base estimator as follows:

In [22]:
!cd .. & ramp_test_submission --submission=starting_kit

Testing Number of air passengers prediction
Reading train and test files from ./data ...
Reading cv ...
Training .\submissions\starting_kit ...
CV fold 0
	train rmse = 0.759
	valid rmse = 0.861
	test rmse = 0.885
CV fold 1
	train rmse = 0.757
	valid rmse = 0.858
	test rmse = 0.879
CV fold 2
	train rmse = 0.74
	valid rmse = 0.858
	test rmse = 0.874
CV fold 3
	train rmse = 0.732
	valid rmse = 0.881
	test rmse = 0.877
CV fold 4
	train rmse = 0.749
	valid rmse = 0.86
	test rmse = 0.887
CV fold 5
	train rmse = 0.743
	valid rmse = 0.84
	test rmse = 0.879
CV fold 6
	train rmse = 0.759
	valid rmse = 0.856
	test rmse = 0.885
CV fold 7
	train rmse = 0.762
	valid rmse = 0.847
	test rmse = 0.876
----------------------------
Mean CV scores
----------------------------
train rmse = 0.75 ± 0.01
valid rmse = 0.858 ± 0.011
test rmse = 0.88 ± 0.004
----------------------------
Bagged scores
----------------------------
valid rmse = 0.841
test rmse = 0.865


<i>To be continued...</i>